In [ ]:
# default_exp core

# Ocetrac

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

We can use this to say hello to anyone at all, but let's start with Hillary.

In [ ]:
say_hello('Hillary')

'Hello Hillary!'

Let's test `say_hello` by passing it a test

In [ ]:
assert say_hello("Hillary")=="Hello Jeremy!"


AssertionError: 

In [ ]:
assert say_hello("Hillary")=="Hello Hillary!"
